# 7章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [ ]:
# 7-2
#!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.10

In [1]:
# 7-3
import random
import glob
import json
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

# 日本語の事前学習モデル
#MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [2]:
#MODEL_NAME="izumi-lab/bert-small-japanese-fin"
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [3]:
# 7-12
# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

In [4]:
# 7-4
class BertForSequenceClassificationMultiLabel(torch.nn.Module):
    
    def __init__(self, model_name, num_labels):
        super().__init__()
        # BertModelのロード
        self.bert = BertModel.from_pretrained(model_name) 
        # 線形変換を初期化しておく
        self.linear = torch.nn.Linear(
            self.bert.config.hidden_size, num_labels
        ) 

    def forward(
        self, 
        input_ids=None, 
        attention_mask=None, 
        token_type_ids=None, 
        labels=None
    ):
        # データを入力しBERTの最終層の出力を得る。
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        last_hidden_state = bert_output.last_hidden_state
        
        # [PAD]以外のトークンで隠れ状態の平均をとる
        averaged_hidden_state = \
            (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True)
        
        # 線形変換
        scores = self.linear(averaged_hidden_state) 
        
        # 出力の形式を整える。
        output = {'logits': scores}

        # labelsが入力に含まれていたら、損失を計算し出力する。
        if labels is not None: 
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
            output['loss'] = loss
            
        # 属性でアクセスできるようにする。
        output = type('bert_output', (object,), output) 

        return output

In [5]:
# 7-13
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters() 
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name, num_labels=num_labels
        ) 

    def training_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)
        scores = output.logits
        labels_predicted = ( scores > 0 ).int()
        num_correct = ( labels_predicted == labels ).all(-1).sum().item()
        accuracy = num_correct/scores.size(0)
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


In [6]:
# 7-14
# 入力する文章
text_list = [
    "今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。",
    "次期の業績の見通しとしましては、売上高につきましては生産卸売事業、直販事業ともに増収を見込んでおります。",
    "農業にマイナス影響あるいは不透明感をもたらす状況が散見されております。"
]

# モデルのロード
#best_model_path = "model/izumi-epoch=4-step=699.ckpt"
best_model_path ="model/tohoku-epoch=3-step=559.ckpt"
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml#.cuda()

# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
#encoding = { k: v.cuda() for k, v in encoding.items() }
encoding = { k: v for k, v in encoding.items() }

# BERTへデータを入力し分類スコアを得る。
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits
labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()

# 結果を表示
for text, label in zip(text_list, labels_predicted):
    print('--')
    print(f'入力：{text}')
    print(f'出力：{label}')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--
入力：今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。
出力：[1, 1]
--
入力：次期の業績の見通しとしましては、売上高につきましては生産卸売事業、直販事業ともに増収を見込んでおります。
出力：[0, 1]
--
入力：農業にマイナス影響あるいは不透明感をもたらす状況が散見されております。
出力：[1, 0]


In [7]:
scores

tensor([[ 4.7287,  3.0522],
        [-7.8847,  6.2935],
        [ 4.0865, -5.9032]])

In [8]:
# 7-14
# 入力する文章
text_list = [
   "今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。",
    "次期の業績の見通しとしましては、売上高につきましては生産卸売事業、直販事業ともに増収を見込んでおります。",
    "農業にマイナス影響あるいは不透明感をもたらす状況が散見されております。"
]
# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)

# BERTへデータを入力し分類スコアを得る。
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits
labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()

# 結果を表示
for text, label in zip(text_list, labels_predicted):
    print('--')
    print(f'入力：{text}')
    print(f'出力：{label}')

--
入力：今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。
出力：[1, 1]
--
入力：次期の業績の見通しとしましては、売上高につきましては生産卸売事業、直販事業ともに増収を見込んでおります。
出力：[0, 1]
--
入力：農業にマイナス影響あるいは不透明感をもたらす状況が散見されております。
出力：[1, 0]


In [57]:
scores

tensor([[ 4.7287,  3.0522],
        [-7.8847,  6.2935],
        [ 4.0865, -5.9032]])

In [ ]:
#[neg,pos]